In [ ]:
#Generación Archivo
#importar librerias
import csv
import numpy as np
import pandas as pd
import random as rd
import string as st
from google.colab import drive
from datetime import datetime
from datetime import timedelta
from google.cloud import storage
from geopy.geocoders import Nominatim

In [ ]:
drive.mount('/content/drive')
path = '/content/drive/MyDrive/Tp_Infra/'
pathXLS = path + 'TP_Final_Especificacion_1.xlsx'
#pathXLS = path + 'TP_Final_Especificacion_1.xlsx'
path_destino = path+'Colab/'
credentials_path = path+'usm-infra-grupo9-6c3307de73cb.json'

storage_client = storage.Client.from_service_account_json(credentials_path)

Mounted at /content/drive


In [ ]:
# Leer el archivo de especificaciones
df_sheets_desc = pd.read_excel(pathXLS, sheet_name=None)
df_cond_vta = df_sheets_desc['Condicion_Venta']
df_unidades = df_sheets_desc['Unidades']
df_tipo_negocio = df_sheets_desc['Tipo_Negocio']
df_cadena = df_sheets_desc['Cadena']
df_dias_visita = df_sheets_desc['Dias_Visita']
df_productos = df_sheets_desc['Productos']
df_cond_venta = df_sheets_desc['Condicion_Venta']
df_localidades = df_sheets_desc['Prov_Localidades']
df_estado = df_sheets_desc['Estado_cliente']
df_dias_visita = df_sheets_desc['Dias_Visita']

In [ ]:
def bucket_exists(bucket_name):

    # Intenta obtener una referencia al bucket
    try:
        bucket = storage_client.get_bucket(bucket_name)
        return True
    except Exception as e:
        if 'Not Found' in str(e):  # Verifica si el error es "No encontrado"
            return False
        else:
            return False

In [ ]:
def generar_codigos_sku(productos):
  codigos_sku = {}
  for producto in productos:
    caracteres = st.ascii_uppercase + st.digits
    sku_codigo = ''.join(rd.choice(caracteres) for _ in range(10))
    codigos_sku[producto] = sku_codigo

  return codigos_sku

In [ ]:
def obtener_latitud_longitud(ciudad, pais):
  geolocator = Nominatim(user_agent="my_geocoder")
  ubicacion = f"{ciudad}, {pais}"
  resultado = geolocator.geocode(ubicacion, timeout=5)
  if resultado is not None:
    return resultado.latitude, resultado.longitude
  else:
    return None

In [ ]:
#generar SKUs a partir de la lista de productos
codigos_sku = generar_codigos_sku(df_productos['nombre'])

#nombre de las columnas de los archivos cliente,venta,stock
columns_v = df_sheets_desc['Desc_Ventas']['Campo']
columns_s = df_sheets_desc['Desc_Stock']['Campo']
columns_m = df_sheets_desc['Desc_Clientes']['Campo']
columns_d = df_sheets_desc['Desc_Deuda']['Campo']

#print(columns_v,columns_s,columns_m,columns_d)

#cantidad de días desde la fecha actual hacia el pasado que generamos datos
cant_dias = 90

#cantidad de dias en el pasado para el alta del cliente(en este cas 2 años desde la fecha altual)
cant_dias_2 = 730

#fecha actual
fecha_actual = datetime.now()

#cantidad de distribuidores
cant_dist = 5

#cantidad de clientes
cant_clientes = 5

#cantidad sucursales
cant_sucursales = 5

for distribuidor in range(1,cant_dist+1):

  for d in range(0,cant_dias):
    dist = [distribuidor for _ in range(cant_clientes)]

    # fecha de cierre
    fech_cierre = fecha_actual - timedelta(days=d)
    fech_cierre = fech_cierre.replace(hour=23, minute=59, second=59)  # Establecer la hora, minutos y segundos fijos
    fech_cierre_string = fech_cierre.strftime('%Y-%m-%d %H-%M-%S')
    fech_cierre_string_2 = fech_cierre.strftime('%Y-%m-%d')
    fech_cie_com_data = [fech_cierre_string for _ in range(cant_clientes)]

    #generamos los datos de venta unidades
    vta_unidades_data = [(rd.randint(0,100)) for _ in range(cant_clientes)]

    #generamos los datos de venta en importes
    vta_importe_data = [round((rd.uniform(100,1000)),2) for _ in range(cant_clientes)]

    #generamos la condicion de venta
    cond_vta_data = [ rd.choice(df_cond_vta['codigo_condicion']) for _ in range(cant_clientes)]

    codigo_sucursal_base = (distribuidor * 100)  # Código base para el distribuidor

    #sucursales del distribuidor
    #el dist1 va a tener sucursales de la 26 a 50
    #el dist2 va a tener sucursales de la 51 a 75
    sucursales_dist = [(codigo_sucursal_base + rd.randint(10, 99)) for _ in range(cant_clientes)]

    #clientes de la sucursal del distribuidor
    clientes = [(codigo_sucursal_base + rd.randint(1000, 9999)) for _ in range(cant_clientes)]

    #Provincia y ciudades
    provincias = [(rd.choice(df_localidades.columns.values)) for _ in range(cant_clientes)]
    ciudades_aleatorias = []
    for _ in range(cant_clientes):
      for p in provincias:
          ciudades_provincia = df_localidades[p].dropna().tolist()
          ciudad_aleatoria = rd.choice(ciudades_provincia)
          ciudades_aleatorias.append(ciudad_aleatoria)

    #Estado
    estado = [(rd.choices(df_estado['nombre'], weights=df_estado['probabilidades'], k=1)[0]) for _ in range(cant_clientes)]

    #Nombre_ cliente
    nombre_cliente = [(f'Cliente_{_}') for _ in range(cant_clientes)]

    #CUIT
    cuit = [(rd.randint(10000000, 99999999) ) for _ in range(cant_clientes)]

    #Razon Social
    razon_social = [(f"Empresa_{_}") for _ in range(cant_clientes)]

    #Direccion
    direccion = [(f"Dirección_{_}") for _ in range(cant_clientes)]

    #Dias de visita
    dias_visita = [(rd.choice(df_dias_visita['codigo_dia'])) for _ in range(cant_clientes)]

    #Telefono
    telefono = [(f"{rd.randint(100, 999)}-{rd.randint(100, 999)}-{rd.randint(1000, 9999)}") for _ in range(cant_clientes)]

    #Fecha de alta y de baja
    fecha_alta = [(fecha_actual - timedelta(days=rd.randint(1, cant_dias_2), hours=rd.randint(0, 23), minutes=rd.randint(0, 59), seconds=rd.randint(0, 59))).strftime('%Y-%m-%d %H:%M:%S') for _ in range(cant_clientes)]
    fecha_alta_datetime = [datetime.strptime(fecha, '%Y-%m-%d %H:%M:%S') for fecha in fecha_alta]

    fecha_baja = []
    for _ in range(cant_clientes):
        if estado[_] == "A":
            fecha_baja.append("")  # Dejar en blanco si el cliente está "activo"
        else:
            fecha_baja_cliente = fecha_alta_datetime[_] + timedelta(days=rd.randint(1, cant_dias_2), hours=rd.randint(0, 23), minutes=rd.randint(0, 59), seconds=rd.randint(0, 59))
            fecha_baja_cliente = min(fecha_baja_cliente, fech_cierre)  # Asegurarse de que la fecha de baja sea menor o igual a la fecha de cierre
            fecha_baja_cliente = max(fecha_baja_cliente, fecha_alta_datetime[_] + timedelta(days=1))  # Asegurarse de que la fecha de baja sea mayor que la fecha de alta
            fecha_baja.append(fecha_baja_cliente.strftime('%Y-%m-%d %H:%M:%S'))

    #Coordenadas
    #agregar coordenada de acuerdo a la ciudad!!!!
    coordenada_latitud = [obtener_latitud_longitud(ciudades_aleatorias[_], 'Argentina')[0] for _ in range(cant_clientes)]
    coordenada_longitud = [obtener_latitud_longitud(ciudades_aleatorias[_], 'Argentina')[1] for _ in range(cant_clientes)]

    #Tipo de negocio
    tipo_negocio = [(rd.choice(df_tipo_negocio['nombre'])) for _ in range(cant_clientes)]

    #ARCHIVO STOCK
    #Unidad
    unidad = [(rd.choice(df_unidades['codigo_unidad'])) for _ in range(cant_clientes)]
    #Stock
    stock = [(rd.randint(100, 500) if unidad == "UNI" else rd.randint(1, 100)) for _ in range(cant_clientes)]


     #ARCHIVO STOCK
    #Unidad
    unidad = [(rd.choice(df_unidades['codigo_unidad'])) for _ in range(cant_clientes)]
    #Stock
    stock = [(rd.randint(100, 500) if unidad == "UNI" else rd.randint(1, 100)) for _ in range(cant_clientes)]

    producto1=[]
    sku_codigo1=[]

    for _ in range(cant_clientes):
      sku_codigo2 = rd.choice(list(codigos_sku.keys())) # elige un random de los codigos_sku
      producto2 = codigos_sku[sku_codigo2]
      sku_codigo1.append(sku_codigo2)
      producto1.append(producto2)

    producto =producto1
    sku_codigo = sku_codigo1

    #DEUDA
    deuda_vencida = [(round(rd.uniform(0, 1000), 2)) for _ in range(cant_clientes)]
    deuda_total = [(round(rd.uniform(0, 10000), 2)) for _ in range(cant_clientes)]

    # Crear Bucket
    bucket_name = 'distribuidor_grupo_9_'+str(distribuidor)

    if bucket_exists(bucket_name):
        bucket = storage_client.get_bucket(bucket_name)
        print(f"El bucket '{bucket_name}' ya existe.")
    else:
        bucket = storage_client.create_bucket(bucket_name)
        #print(f"Bucket {bucket.name} creado exitosamente.")


    #data frame con data
    venta_clientes_data = list(zip(dist,sucursales_dist,clientes,fech_cie_com_data,sku_codigo,vta_unidades_data,vta_importe_data,cond_vta_data))

    stock_data = list(zip(dist,sucursales_dist, fech_cie_com_data, sku_codigo, producto, stock, unidad))

    clientes_data = list(zip(dist,sucursales_dist, clientes, ciudades_aleatorias, provincias, estado, nombre_cliente, cuit, razon_social, direccion, dias_visita, telefono, fecha_alta, fecha_baja, coordenada_latitud, coordenada_longitud, cond_vta_data, deuda_vencida, tipo_negocio))

    deuda_data = list(zip(dist,sucursales_dist,clientes, fech_cie_com_data,deuda_vencida,deuda_total ))

    df_vta_cli = pd.DataFrame(venta_clientes_data,columns=columns_v)
    df_stock = pd.DataFrame(stock_data,columns=columns_s)
    df_clientes = pd.DataFrame(clientes_data,columns=columns_m)
    df_deuda =  pd.DataFrame(deuda_data,columns=columns_d)

    #upload file al bucket
    carpeta_destino =  path_destino + bucket_name

    archivo_destino_v =  'venta_'+fech_cierre_string_2+'.csv'
    archivo_destino_s =  'stock_'+fech_cierre_string_2+'.csv'
    archivo_destino_m =  'cliente_'+fech_cierre_string_2+'.csv'
    archivo_destino_d =  'deuda_'+fech_cierre_string_2+'.csv'

    df_vta_cli.to_csv(archivo_destino_v, encoding='utf-8', index=False)
    df_stock.to_csv(archivo_destino_s, encoding='utf-8', index=False)
    df_clientes.to_csv(archivo_destino_m, encoding='utf-8', index=False)
    df_deuda.to_csv(archivo_destino_d, encoding='utf-8', index=False)

    blob_v = bucket.blob(archivo_destino_v)
    blob_s = bucket.blob(archivo_destino_s)
    blob_m = bucket.blob(archivo_destino_m)
    blob_d = bucket.blob(archivo_destino_d)

    blob_v.upload_from_string(df_vta_cli.to_csv(index=False), content_type='text/csv')
    blob_s.upload_from_string(df_stock.to_csv(index=False), content_type='text/csv')
    blob_m.upload_from_string(df_clientes.to_csv(index=False), content_type='text/csv')
    blob_d.upload_from_string(df_deuda.to_csv(index=False), content_type='text/csv')



El bucket 'distribuidor_grupo_9_1' ya existe.
El bucket 'distribuidor_grupo_9_1' ya existe.
El bucket 'distribuidor_grupo_9_1' ya existe.
El bucket 'distribuidor_grupo_9_1' ya existe.
El bucket 'distribuidor_grupo_9_1' ya existe.
El bucket 'distribuidor_grupo_9_1' ya existe.
El bucket 'distribuidor_grupo_9_1' ya existe.
El bucket 'distribuidor_grupo_9_1' ya existe.
El bucket 'distribuidor_grupo_9_1' ya existe.
El bucket 'distribuidor_grupo_9_1' ya existe.
El bucket 'distribuidor_grupo_9_1' ya existe.
El bucket 'distribuidor_grupo_9_1' ya existe.
El bucket 'distribuidor_grupo_9_1' ya existe.
El bucket 'distribuidor_grupo_9_1' ya existe.
El bucket 'distribuidor_grupo_9_1' ya existe.
El bucket 'distribuidor_grupo_9_1' ya existe.
El bucket 'distribuidor_grupo_9_1' ya existe.
El bucket 'distribuidor_grupo_9_1' ya existe.
El bucket 'distribuidor_grupo_9_1' ya existe.
El bucket 'distribuidor_grupo_9_1' ya existe.
El bucket 'distribuidor_grupo_9_1' ya existe.
El bucket 'distribuidor_grupo_9_1'

El bucket 'distribuidor_grupo_9_5' ya existe.
El bucket 'distribuidor_grupo_9_5' ya existe.
El bucket 'distribuidor_grupo_9_5' ya existe.
El bucket 'distribuidor_grupo_9_5' ya existe.
El bucket 'distribuidor_grupo_9_5' ya existe.
El bucket 'distribuidor_grupo_9_5' ya existe.
El bucket 'distribuidor_grupo_9_5' ya existe.
El bucket 'distribuidor_grupo_9_5' ya existe.
El bucket 'distribuidor_grupo_9_5' ya existe.
El bucket 'distribuidor_grupo_9_5' ya existe.
El bucket 'distribuidor_grupo_9_5' ya existe.
El bucket 'distribuidor_grupo_9_5' ya existe.
El bucket 'distribuidor_grupo_9_5' ya existe.
El bucket 'distribuidor_grupo_9_5' ya existe.
El bucket 'distribuidor_grupo_9_5' ya existe.
El bucket 'distribuidor_grupo_9_5' ya existe.
El bucket 'distribuidor_grupo_9_5' ya existe.
El bucket 'distribuidor_grupo_9_5' ya existe.
El bucket 'distribuidor_grupo_9_5' ya existe.
El bucket 'distribuidor_grupo_9_5' ya existe.
El bucket 'distribuidor_grupo_9_5' ya existe.
El bucket 'distribuidor_grupo_9_5'